In [2]:
!pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.71.1
    Uninstalling google-cloud-aiplatform-1.71.1:
      Successfully uninstalled google-cloud-aiplatform-1.71.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==1.71.1, but you have google-cloud-aiplatform 1.87.0 which is incompatible.


In [3]:
!pip install --upgrade vertexai

  Using cached google_cloud_aiplatform-1.71.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached google_cloud_aiplatform-1.71.1-py2.py3-none-any.whl (6.2 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.87.0
    Uninstalling google-cloud-aiplatform-1.87.0:
      Successfully uninstalled google-cloud-aiplatform-1.87.0


In [4]:
!pip install google-cloud-storage

In [5]:
!pip install nltk

In [6]:
!pip install pydub

In [7]:

!pip install  google-cloud-speech

In [8]:
import os
from pydub import AudioSegment
from google.cloud import speech
from google.cloud import storage

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting, Part


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/thermal-origin-454105-s5-c6291f413f44.json"

In [10]:


def transcribe_mp3_to_text(mp3_path, bucket_name, language_code):


    wav_path = "/content/converted_audio.wav"
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(wav_path, format="wav")

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    destination_blob_name = "converted_audio.wav"
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(wav_path)

    client = speech.SpeechClient()

    audio_gcs = speech.RecognitionAudio(uri=f"gs://{bucket_name}/{destination_blob_name}")
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code=language_code,
        alternative_language_codes=["zh-CN", "fr-FR", "ja-JP", "ru-RU", "ar-SA"],  # 增加俄语和阿拉伯语
        enable_automatic_punctuation=True
    )

    operation = client.long_running_recognize(config=config, audio=audio_gcs)

    response = operation.result(timeout=300)

    transcriptions = []
    language = None
    for result in response.results:
        transcript = result.alternatives[0].transcript
        language = result.language_code
        transcriptions.append(transcript)

    combined_transcription = " ".join(transcriptions)

    return f"{language}\n{combined_transcription}"

def transcribe(mp3_path, bucket_name="my-speech-bucket1ze", language_code="en-US"):
    return transcribe_mp3_to_text(mp3_path, bucket_name, language_code)




In [11]:
def split_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

In [12]:

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

def ask_question(text: str):
    vertexai.init(
        project="829020429280",
        location="us-central1",
        api_endpoint="us-central1-aiplatform.googleapis.com"
    )

    model = GenerativeModel(
        "projects/829020429280/locations/us-central1/endpoints/6002987141494210560",
    )

    chat = model.start_chat()

    response = chat.send_message(
        text,
        generation_config=generation_config,
        safety_settings=safety_settings
    )


    return response.text





In [15]:

def process_and_ask(mp3_path):
    transcribed_text = transcribe(mp3_path)

    transcription_content = transcribed_text.split("\n")[1]

    sentences = split_into_sentences(transcription_content)

    prompt_text = "\n\nPlease determine whether this sentence is toxic; if it is, output 'toxic', if it is not, output 'non-toxic':\n\n"

    responses = []
    is_toxic = False

    output_text = []

    for i, sentence in enumerate(sentences, 1):

        question_text = sentence + prompt_text


        response = ask_question(question_text)

        if not response or response.strip() == "":
            continue

        response = response.strip()

        response_dict = {
            "sentence": sentence,
            "response": response
        }
        responses.append(response_dict)

        if response.lower() == 'toxic':
            is_toxic = True

        output_text.append(f"sentence {i}: {sentence}")
        output_text.append(f"response {i}: {response}")
        output_text.append("\n")

    if is_toxic:
        overall_response = "toxic"
    else:
        overall_response = "non-toxic"

    transcription_lan = transcribed_text.split("\n")[0]
    print(f"lan：{transcription_lan}")
    print(f"result：{overall_response}")

    with open("responses_output.txt", "w", encoding="utf-8") as file:
        file.write("\n".join(output_text))
        file.write(f"result: {overall_response}")



In [14]:
mp3_path = "/content/HIGH ARABIC SCRIPT Civilizing the Indian.mp3"
process_and_ask(mp3_path)

lan：ar-x-gulf
result：toxic
